In [1]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import math
import matplotlib.pyplot as plt

In [2]:
def get_hash(fpath, hash_length):
    dim = int(math.sqrt(hash_length))+1    
    r_str=''    
    img=cv2.imread(fpath,0)        # read image as gray scale image
    img=cv2.resize(img, (dim,dim), interpolation = cv2.INTER_NEAREST)
    img=img.flatten()
    list2=list(img) 
    for col in range (0,len(list2)-1):
        if(list2[col]>list2[col+1]):
            value=str(1)
        else:
            value=str(0)
        r_str=r_str + value   
    return r_str

In [3]:
def match(value1, value2, difference):
    length=len(value1)
    mismatch=0
    for i in range(0, length):
        if value1[i] !=value2[i]:
            mismatch = mismatch  + 1
            if mismatch>difference:
                return False
    return True

In [4]:
def find_duplicates(hash_dict, difference):
    dup_dict={}
    key_list=[]
    for key,value in tqdm(hash_dict.items()):
        dup_list=[]
        for key2,value2 in hash_dict.items():
            if key != key2 and key not in key_list and key2 not in key_list:
                if match(value,value2,difference):
                    dup_list.append(key2)
                    dup_dict[key]=dup_list
                    if key2 not in key_list:
                        key_list.append(key2)                    
        if key not in key_list:
            key_list.append(key)
        
        #print('key=

In [5]:
def find_duplicate(source_dir,distance):
    sense_list=[0, 3,6,9,12,21,27,32,45,64] 
    difference=sense_list[sensitivity] # hash length is 256 if there are less than distance differences in the hash
    # then images are considered duplicates
    hash_length= 256
    f_list=[] # will be a list of files ( full path) that are not duplicates of each other
    duplicate_files_list = [] # will contain a list of duplicate filepaths
    dup_list =[] # list of files that are duplicates of files in f_list
    class_list= os.listdir(source_dir)    
    for klass in class_list: # iterate through classes 
        class_path=os.path.join (source_dir, klass)
        if os.path.isdir(class_path):
            flist=os.listdir(class_path)
            for f in flist:
                fpath=os.path.join(class_path,f)
                f_list.append(fpath) # list of all filepaths in the source_dir
        else:
            for f in class_list:
                fpath=os.path.join(source_dir,f)
                f_list.append(fpath)
            break
    hash_dict={}
    for fpath in tqdm(f_list): # iterate through the list of filepaths
        hash_dict[fpath]=get_hash(fpath,hash_length)  # store the hash in a dictionary with key-filepath and value=hash
    # now go the dictionary and create a new dictionary of duplicate files
    # dup_dict the key is a fileppath and the values are filepaths of images that are duplicates of the key filepath
    dup_dict=find_duplicates(hash_dict, difference)
    if len(dup_dict)<1:
        print('No duplicate images were found in ', source_dir, '\n')            
    else:
        dup_count = 0
        for key, values in dup_dict.items():
            dup_count += len(values) 
            for fpath in values:
                f_list.remove(fpath)
                duplicate_files_list.append(fpath)
                
        print('\n', dup_count, ' duplicate images were detected and removed from the list of good_files')
        #print (dup_dict)
    return f_list, duplicate_files_list  